In [24]:
%matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 17: Pyroclastic Flow ---</h2><p>Your handheld device has located an alternative exit from the cave for you and the elephants.  The ground is rumbling almost continuously now, but the strange valves bought you some time. It's definitely getting warmer in here, though.</p>
<p>The tunnels eventually open into a very tall, narrow chamber. Large, oddly-shaped rocks are falling into the chamber from above, presumably due to all the rumbling. If you can't work out where the rocks will fall next, you might be <span title="I am the man who arranges the blocks / that descend upon me from up above!">crushed</span>!</p>
<p>The five types of rocks have the following peculiar shapes, where <code>#</code> is rock and <code>.</code> is empty space:</p>
<pre><code>####

.#.

###

.#.

..#
..#

###

#

#

#

#

##

##

</code></pre>

<p>The rocks fall in the order shown above: first the <code>-</code> shape, then the <code>+</code> shape, and so on. Once the end of the list is reached, the same order repeats: the <code>-</code> shape falls first, sixth, 11th, 16th, etc.</p>
<p>The rocks don't spin, but they do get pushed around by jets of hot gas coming out of the walls themselves. A quick scan reveals the effect the jets of hot gas will have on the rocks as they fall (your puzzle input).</p>
<p>For example, suppose this was the jet pattern in your cave:</p>
<pre><code>&gt;&gt;&gt;&lt;&lt;&gt;&lt;&gt;&gt;&lt;&lt;&lt;&gt;&gt;&lt;&gt;&gt;&gt;&lt;&lt;&lt;&gt;&gt;&gt;&lt;&lt;&lt;&gt;&lt;&lt;&lt;&gt;&gt;&lt;&gt;&gt;&lt;&lt;&gt;&gt;
</code></pre>
<p>In jet patterns, <code>&lt;</code> means a push to the left, while <code>&gt;</code> means a push to the right. The pattern above means that the jets will push a falling rock right, then right, then right, then left, then left, then right, and so on. If the end of the list is reached, it repeats.</p>
<p>The tall, vertical chamber is exactly <em>seven units wide</em>. Each rock appears so that its left edge is two units away from the left wall and its bottom edge is three units above the highest rock in the room (or the floor, if there isn't one).</p>
<p>After a rock appears, it alternates between <em>being pushed by a jet of hot gas</em> one unit (in the direction indicated by the next symbol in the jet pattern) and then <em>falling one unit down</em>. If any movement would cause any part of the rock to move into the walls, floor, or a stopped rock, the movement instead does not occur. If a <em>downward</em> movement would have caused a falling rock to move into the floor or an already-fallen rock, the falling rock stops where it is (having landed on something) and a new rock immediately begins falling.</p>
<p>Drawing falling rocks with <code>@</code> and stopped rocks with <code>#</code>, the jet pattern in the example above manifests as follows:</p>
<pre><code>The first rock begins falling:
|..@@@@.|
|.......|
|.......|
|.......|
+-------+

Jet of gas pushes rock right:
|...@@@@|
|.......|
|.......|
|.......|
+-------+

Rock falls 1 unit:
|...@@@@|
|.......|
|.......|
+-------+

Jet of gas pushes rock right, but nothing happens:
|...@@@@|
|.......|
|.......|
+-------+

Rock falls 1 unit:
|...@@@@|
|.......|
+-------+

Jet of gas pushes rock right, but nothing happens:
|...@@@@|
|.......|
+-------+

Rock falls 1 unit:
|...@@@@|
+-------+

Jet of gas pushes rock left:
|..@@@@.|
+-------+

Rock falls 1 unit, causing it to come to rest:
|..####.|
+-------+

A new rock begins falling:
|...@...|
|..@@@..|
|...@...|
|.......|
|.......|
|.......|
|..####.|
+-------+

Jet of gas pushes rock left:
|..@....|
|.@@@...|
|..@....|
|.......|
|.......|
|.......|
|..####.|
+-------+

Rock falls 1 unit:
|..@....|
|.@@@...|
|..@....|
|.......|
|.......|
|..####.|
+-------+

Jet of gas pushes rock right:
|...@...|
|..@@@..|
|...@...|
|.......|
|.......|
|..####.|
+-------+

Rock falls 1 unit:
|...@...|
|..@@@..|
|...@...|
|.......|
|..####.|
+-------+

Jet of gas pushes rock left:
|..@....|
|.@@@...|
|..@....|
|.......|
|..####.|
+-------+

Rock falls 1 unit:
|..@....|
|.@@@...|
|..@....|
|..####.|
+-------+

Jet of gas pushes rock right:
|...@...|
|..@@@..|
|...@...|
|..####.|
+-------+

Rock falls 1 unit, causing it to come to rest:
|...#...|
|..###..|
|...#...|
|..####.|
+-------+

A new rock begins falling:
|....@..|
|....@..|
|..@@@..|
|.......|
|.......|
|.......|
|...#...|
|..###..|
|...#...|
|..####.|
+-------+
</code></pre>

<p>The moment each of the next few rocks begins falling, you would see this:</p>
<pre><code>|..@....|
|..@....|
|..@....|
|..@....|
|.......|
|.......|
|.......|
|..#....|
|..#....|
|####...|
|..###..|
|...#...|
|..####.|
+-------+

|..@@...|
|..@@...|
|.......|
|.......|
|.......|
|....#..|
|..#.#..|
|..#.#..|
|#####..|
|..###..|
|...#...|
|..####.|
+-------+

|..@@@@.|
|.......|
|.......|
|.......|
|....##.|
|....##.|
|....#..|
|..#.#..|
|..#.#..|
|#####..|
|..###..|
|...#...|
|..####.|
+-------+

|...@...|
|..@@@..|
|...@...|
|.......|
|.......|
|.......|
|.####..|
|....##.|
|....##.|
|....#..|
|..#.#..|
|..#.#..|
|#####..|
|..###..|
|...#...|
|..####.|
+-------+

|....@..|
|....@..|
|..@@@..|
|.......|
|.......|
|.......|
|..#....|
|.###...|
|..#....|
|.####..|
|....##.|
|....##.|
|....#..|
|..#.#..|
|..#.#..|
|#####..|
|..###..|
|...#...|
|..####.|
+-------+

|..@....|
|..@....|
|..@....|
|..@....|
|.......|
|.......|
|.......|
|.....#.|
|.....#.|
|..####.|
|.###...|
|..#....|
|.####..|
|....##.|
|....##.|
|....#..|
|..#.#..|
|..#.#..|
|#####..|
|..###..|
|...#...|
|..####.|
+-------+

|..@@...|
|..@@...|
|.......|
|.......|
|.......|
|....#..|
|....#..|
|....##.|
|....##.|
|..####.|
|.###...|
|..#....|
|.####..|
|....##.|
|....##.|
|....#..|
|..#.#..|
|..#.#..|
|#####..|
|..###..|
|...#...|
|..####.|
+-------+

|..@@@@.|
|.......|
|.......|
|.......|
|....#..|
|....#..|
|....##.|
|##..##.|
|######.|
|.###...|
|..#....|
|.####..|
|....##.|
|....##.|
|....#..|
|..#.#..|
|..#.#..|
|#####..|
|..###..|
|...#...|
|..####.|
+-------+
</code></pre>

<p>To prove to the elephants your simulation is accurate, they want to know how tall the tower will get after 2022 rocks have stopped (but before the 2023rd rock begins falling). In this example, the tower of rocks will be <code><em>3068</em></code> units tall.</p>
<p><em>How many units tall will the tower of rocks be after 2022 rocks have stopped falling?</em></p>
</article>


In [25]:
from collections import deque
from copy import copy
from itertools import cycle

from more_itertools import minmax


tests = [
    {
        "name": "Example",
        "s": ">>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>",
        "stop_after": 2022,
        "expected": 3068,
    },
]


class Rock:
    rocks_str = "####\n\n.#.\n###\n.#.\n\n..#\n..#\n###\n\n#\n#\n#\n#\n\n##\n##"

    def __init__(self, s: str, chamber: Chamber, _print=False) -> None:
        self.blocks = frozenset(
            (y, x)
            for y, row in enumerate(s.splitlines())
            for x, v in enumerate(row)
            if v == "#"
        )
        self.x, self.width = minmax(x for _, x in self.blocks)
        self.y, self.height = minmax(y for y, _ in self.blocks)
        self.width = self.width - self.x + 1
        self.height = self.height - self.y + 1
        self.chamber = chamber
        self.print = _print

    def move(self) -> None:
        self.place()

        while True:
            wind = next(self.chamber.winds)
            self.move_horizontal(wind)

            if not self.move_vertical_successfully():
                self.freeze()
                return

    def freeze(self) -> None:
        for dyy, dxx in self.blocks:
            self.chamber.chamber[self.y + dyy][self.x + dxx] = "."

        for dyy, dxx in self.blocks:
            self.chamber.chamber[self.y + dyy][self.x + dxx] = "#"

        if self.print:
            print(f"Rock falls 1 unit, causing it to come to rest:")
            print(self.chamber)
            print()

    def place(self) -> None:
        self.x = self.chamber.left + self.chamber.dx

        first = len(self.chamber.chamber) == 0

        for y in range(self.chamber.dy + self.y + self.height - 1, -1, -1):
            self.chamber.chamber.appendleft(
                [
                    "@" if self.in_rock(x, y) else "."
                    for x in range(self.chamber.left, self.chamber.right)
                ]
            )

        if self.print:
            print(
                "The first rock begins falling:"
                if first
                else "A new rock begins falling:"
            )
            print(self.chamber)
            print()

    def move_vertical_successfully(self, dy=1) -> bool:
        if self.vertical_collide(dy):
            return False

        chamber = self.chamber.chamber

        if all(
            chamber[self.y + self.height - 1 + dy][x] == "."
            for x in range(self.chamber.left, self.chamber.right)
        ):
            chamber.rotate(len(chamber) - self.height)
            chamber.popleft()
            chamber.rotate(self.height - len(chamber))
        else:
            for dyy, dxx in self.blocks:
                chamber[self.y + dyy][self.x + dxx] = "."

            self.y += dy

            for dyy, dxx in self.blocks:
                chamber[self.y + dyy][self.x + dxx] = "@"

            while all(c == "." for c in chamber[0]):
                chamber.popleft()
                self.y -= 1

        if self.print:
            print("Rock falls 1 unit:")
            print(self.chamber)
            print()

        return True

    def vertical_collide(self, dy: int) -> bool:
        chamber = self.chamber.chamber

        if self.y + self.height - 1 + dy >= len(chamber):
            return True

        return any(
            chamber[self.y + dyy + dy][self.x + dxx] == "#" for dyy, dxx in self.blocks
        )

    def move_horizontal(self, wind: str) -> None:
        chamber = self.chamber.chamber
        dx = 1 if wind == ">" else -1

        if not self.horizontal_collide(dx):
            for dyy, dxx in self.blocks:
                chamber[self.y + dyy][self.x + dxx] = "."

            self.x += dx

            for dyy, dxx in self.blocks:
                chamber[self.y + dyy][self.x + dxx] = "@"

            if self.print:
                print(f"Jet of gas pushes rock {'right' if wind=='>' else 'left'}:")
                print(self.chamber)
                print()

        elif self.print:
            print(
                f"Jet of gas pushes rock {'right' if wind=='>' else 'left'}, but nothing happens:"
            )
            print(self.chamber)
            print()

    def horizontal_collide(self, dx: int) -> bool:
        if self.x + dx < self.chamber.left:
            return True
        if self.x + dx + self.width > self.chamber.right:
            return True

        return any(
            self.chamber.chamber[self.y + dyy][self.x + dxx + dx] == "#"
            for dyy, dxx in self.blocks
        )

    def in_rock(self, x: int, y: int) -> bool:
        return (y - self.y, x - self.x) in self.blocks

    def __str__(self) -> str:
        return f"Rock(x={self.x}, y={self.y}, blocks={self.blocks})"

    __repr__ = __str__

    @classmethod
    def create_rocks(cls, chamber: Chamber, _print=False) -> tuple[Rock, ...]:
        return tuple(cls(r, chamber, _print) for r in re.split(r"\n\n", cls.rocks_str))


class Chamber:
    left = 0
    right = 7
    dx = 2
    dy = 3

    def __init__(self, s: str, _print=False) -> None:
        self.highest_rock = 0
        self.rocks = cycle(Rock.create_rocks(self, _print))
        self.winds = cycle(s.strip())
        self.chamber = deque([])

    def simulate(self, stop_after: int) -> int:
        for _ in range(stop_after):
            rock = copy(next(self.rocks))
            rock.move()
        return len(self.chamber)

    def __str__(self) -> str:
        return "\n".join(
            (
                "\n".join(f"|{''.join(l)}|" for l in self.chamber),
                f"+{'-'*(self.right-self.left)}+",
            )
        )

    __repr__ = __str__


@test(tests=tests)
def test_part_I(s: str, stop_after: int) -> int:
    c = Chamber(s)
    return c.simulate(stop_after)


Test Example passed, for test_part_I.
Success


In [26]:
with open("../input/day17.txt") as quotient:
    puzzle = quotient.read()

print(f"Part I: {Chamber(puzzle).simulate(2022)}")

Part I: 3048


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>3048</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The elephants are not impressed by your simulation. They demand to know how tall the tower will be after <code><em>1000000000000</em></code> rocks have stopped! Only then will they feel confident enough to proceed through the cave.</p>
<p>In the example above, the tower would be <code><em>1514285714288</em></code> units tall!</p>
<p><em>How tall will the tower be after <code>1000000000000</code> rocks have stopped?</em></p>
</article>


In [ ]:
from pprint import pprint
from typing import override


tests = [
    {
        "name": "Example 1",
        "s": ">>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>",
        "stop_after": 2022,
        "expected": 3068,
    },
    {
        "name": "Example 2",
        "s": puzzle,
        "stop_after": 2022,
        "expected": 3048,
    },
    {
        "name": "Example 3",
        "s": ">>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>",
        "stop_after": 1_000_000_000_000,
        "expected": 1_514_285_714_288,
    },
]


class RockII(Rock):

    def freeze(self) -> None:
        super().freeze()

        chamber = self.chamber.chamber
        seen = self.chamber.seen_when
        count = self.chamber.count

        h = "\n".join(
            "".join(chamber[self.y + dyy]) for dyy in {y for y, x in self.blocks}
        )

        seen[h].add(count)

        self.chamber.count += 1


class ChamberII(Chamber):
    def __init__(self, s, _print=False) -> None:
        super().__init__(s, _print)
        self.rocks = cycle(RockII.create_rocks(self, _print))
        self.count = 0
        self.seen_when = defaultdict(set)

    @classmethod
    def simulateII(cls, s: str, stop_after: int) -> int:
        ch = cls(s)
        res = ch.simulate(len(s.strip()) * len(RockII.create_rocks(ch)))

        max_period, start = cls.get_max_periodicity(ch)

        quotient, remainder = divmod(stop_after - start, max_period)

        ch = cls(s)
        h1 = ch.simulate(start)
        h2 = ch.simulate(max_period)
        h3 = ch.simulate(remainder)

        res = h1
        res += quotient * (h2 - h1)
        res += h3 - h2
        return res

    @classmethod
    def get_max_periodicity(cls, ch) -> tuple[int, int]:
        max_period = -1
        start = -1
        for k, v in ch.seen_when.items():
            v = sorted(v)
            if len(v) > 3 and all(
                v[i] - v[i - 1] == v[i - 1] - v[i - 2] for i in range(2, len(v))
            ):
                if v[1] - v[0] > max_period:
                    max_period = v[1] - v[0]
                    start = v[0]
        return max_period, start


@test(tests=tests)
def test_part_II(s: str, stop_after: int) -> int:
    res = ChamberII.simulateII(s, stop_after)
    return res


Test Example 1 passed, for test_part_II.
Test Example 2 passed, for test_part_II.
Test Example 3 passed, for test_part_II.
Success


In [29]:
print(f"Part II: {ChamberII.simulateII(puzzle, 1_000_000_000_000):,}")

Part II: 1,504,093,567,249


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>1504093567249</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
